<a href="https://colab.research.google.com/github/saravanamuthu1/sentiment_analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from collections import Counter
import math
import re

In [94]:
class Naive_bayes:
  def __init__(self) -> None:
    self.mylist=[]
    self.li=[]
    self.V_freq = None
    self.freq_pos=None
    self.freq_neg=None
    self.df=None
    self.total_porobaility_for_pos=0
    self.total_porobaility_for_neg=0
  def get_data_set(self,my_data):
    for line in my_data:
      value=line.rstrip("\n").split("\t")
      self.mylist.append(line.rstrip("\n").split("\t"))
    df1=pd.DataFrame(self.mylist)
    df1.columns=["label","tweet"]
    return df1,self.mylist
  #fit function
  def fit(self,df1):
    df1['train_value'] = df1.train_value.apply(lambda x:re.sub(r'[^\w\s]', '', x))
    df1['train_value'] = df1.train_value.apply(lambda x:x.strip())
    df1['train_value'] =df1.train_value.apply(lambda x:x.replace("USER", ""))
    df1["train_value"]= df1.train_value.apply(lambda x:" ".join(x.split()))
    df1["train_value"]= df1.train_value.apply(lambda x:''.join([i for i in x if not i.isdigit()]))
    self.V_freq = Counter(" ".join(df1['train_value'].values.tolist()).split(" "))
    self.freq_pos= Counter(" ".join(df1.loc[df1.train_label=='1']['train_value'].values.tolist()).split(" ")) 
    self.freq_neg = Counter(" ".join(df1.loc[df1.train_label=='0']['train_value'].values.tolist()).split(" ")) 
    for i in self.V_freq:
      if i in self.freq_pos and i in self.freq_neg:
        self.li.append(i)
    for i in self.li:
      self.V_freq[i] = 1
      self.freq_pos[i] = 1
      self.freq_neg[i] = 1
    self.total_porobaility_for_pos=(len(self.freq_pos)/len(self.V_freq))
    self.total_porobaility_for_neg=(len(self.freq_neg)/len(self.V_freq))
    for i in self.V_freq:
      if i in self.freq_pos and i in self.freq_neg:
        self.li.append(i)

  # data pre-processing
  def pre_processing(self,mytest_file):
    mytest_file['train_value'] = mytest_file.train_value.apply(lambda x:re.sub(r'[^\w\s]', '', x))
    mytest_file['train_value'] = mytest_file.train_value.apply(lambda x:x.strip())
    mytest_file['train_value'] =mytest_file.train_value.apply(lambda x:x.replace("USER", ""))
    mytest_file["train_value"]= mytest_file.train_value.apply(lambda x:" ".join(x.split()))
    mytest_file["train_value"]= mytest_file.train_value.apply(lambda x:''.join([i for i in x if not i.isdigit()]))
    return mytest_file
  def pre_processing_test(self,mytest_file):
    mytest_file['test_value'] = mytest_file.test_value.apply(lambda x:re.sub(r'[^\w\s]', '', x))
    mytest_file['test_value'] = mytest_file.test_value.apply(lambda x:x.strip())
    mytest_file['test_value'] =mytest_file.test_value.apply(lambda x:x.replace("USER", ""))
    mytest_file["test_value"]= mytest_file.test_value.apply(lambda x:" ".join(x.split()))
    mytest_file["test_value"]= mytest_file.test_value.apply(lambda x:''.join([i for i in x if not i.isdigit()]))
    return mytest_file
  # predict fucntion
  def predict(self,mytxt):
    p_w_pos_value =0.0
    p_w_neg_value =0.0
    mytxt=mytxt.split()
    mytxt=list(set(mytxt))
    pos_score = self.total_porobaility_for_pos
    neg_score = self.total_porobaility_for_neg
    for w in mytxt:
      if w in self.freq_pos:
        p_w_pos_value=((1+(self.freq_pos.get(w,0)))/((2*1)+len(self.freq_pos)))
      if w in self.freq_neg:
        p_w_neg_value = ((1+(self.freq_neg.get(w,0)))/((2*1)+len(self.freq_neg)))
      pos_score= pos_score*p_w_pos_value
      neg_score= neg_score*p_w_neg_value
    max_value=max(pos_score,neg_score)
    if max_value == pos_score:
      return 1
    else:
      return 0   


if __name__ == "__main__":
  #to fit the data
  testfile = open('train-v2.tsv', 'r')
  obj1= Naive_bayes()
  df1,total_data=obj1.get_data_set(testfile)
  # predict values
  # upload your dataset in the colab and change the name here to predict
  predict_file= open('train-v2.tsv', 'r')
  #since we don't have test set we have used split the entire corpus. we can alter still alter the program for jsut test set alone.
  train_set,test_set=total_data[0:60000],total_data[60000:80000]
  df2=pd.DataFrame(train_set)
  df2.columns=["train_label","train_value"]
  df2=obj1.pre_processing(df2)
  obj1.fit(df2)
  df2['train_label'] = df2.train_label.apply(lambda x:int(x))
  df3=pd.DataFrame(test_set)
  df3.columns=["test_label","test_value"]
  df3=obj1.pre_processing_test(df3)
  df2['total_prediction_train']=df2.train_value.apply(lambda x:obj1.predict(x))
  df3['total_prediction_test']=df3.test_value.apply(lambda x:obj1.predict(x))
  df3['test_label'] = df3.test_label.apply(lambda x:int(x))
  accuracy1 = accuracy_score(df2['train_label'], df2['total_prediction_train'])
  accuracy2 = accuracy_score(df3['test_label'], df3['total_prediction_test'])
  print("accuracy on the trianing_set:",accuracy1)
  print("accuracy on the t_set:",accuracy2)




  

    


  

accuracy on the trianing_set: 0.64465
accuracy on the t_set: 0.5361
